In [1]:
import numpy as np
import tensorflow as tf
# tf.compat.v1.disable_eager_execution()
tf.get_logger().setLevel(40)
tf.compat.v1.disable_v2_behavior()
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt

import data_load
import data_process
import train_model

2023-08-30 10:18:35.545265: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 10:18:36.168608: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from sklearn import preprocessing

DATA_PATH = "data/Turbine_Data_Penmanshiel_11_2021-01-01_-_2021-07-01_1051.csv"
LOGS_PATH = "data/Status_Penmanshiel_11_2021-01-01_-_2021-07-01_1051.csv"

data = data_load.load_turbine_data_without_time(DATA_PATH)
logs = data_load.load_turbine_logs_with_endtime(LOGS_PATH)
labels = data_load.get_data_label(data_load.load_turbine_data(DATA_PATH), logs)
scaler = preprocessing.StandardScaler()
data_imputer = KNNImputer(n_neighbors=10)
data = data_process.impute_data(data)
data = data_process.standardize_data(data, scaler=scaler)
print(data.shape)
data = np.array(data, dtype='float32')

(26064, 299)


In [3]:
from sklearn.model_selection import train_test_split

epochs = 3000
k = 10

i = 0
inner_size = 20
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1)
    # ae_model = model.RnnAutoEncoder(input_size=np.size(data, axis=1)).to(device)
    # loss = nn.MSELoss()
ae_model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(np.size(data, axis=1),)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(inner_size, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(np.size(data, axis=1))
])
X_train_raw = X_train
X_train = X_train[y_train == 0]
# optimizer = torch.optim.Adam(ae_model.parameters(), lr=0.001)

ae_model.compile(optimizer='adam', loss='mse', metrics=['mean_squared_error'])

# X_train, X_test = data[train_index], data[test_index]

ae_model.fit(X_train, X_train, epochs=epochs, batch_size=np.size(data, axis=0))
out = ae_model(X_test)

# tf.compat.v1.initialize_all_variables().run(session=tf.compat.v1.Session())
# test_loss, res = ae_model.evaluate(X_test, X_test, batch_size=np.size(data, axis=0))
# X_pred = np.array(out.eval(session=tf.compat.v1.Session()))
X_pred = ae_model(X_test)
res = tf.math.reduce_sum(tf.math.pow(X_pred - X_test, 2), 1)


    # res = train_model.autoencoder_test(X_test, X_test, ae_model, 1000)
    # break

Train on 21865 samples


2023-08-30 10:19:04.473768: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-30 10:19:04.512981: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-30 10:19:04.513441: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-30 10:19:04.515413: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-30 10:19:04.515889: I tensorflow/compile

Epoch 1/3000
21865/21865 [==============================] - 1s 32us/sample - loss: 0.7734 - mean_squared_error: 0.7734
Epoch 2/3000
21865/21865 [==============================] - 0s 2us/sample - loss: 0.7695 - mean_squared_error: 0.7695
Epoch 3/3000
21865/21865 [==============================] - 0s 2us/sample - loss: 0.7670 - mean_squared_error: 0.7670
Epoch 4/3000
21865/21865 [==============================] - 0s 2us/sample - loss: 0.7637 - mean_squared_error: 0.7637
Epoch 5/3000
21865/21865 [==============================] - 0s 1us/sample - loss: 0.7589 - mean_squared_error: 0.7589
Epoch 6/3000
21865/21865 [==============================] - 0s 2us/sample - loss: 0.7519 - mean_squared_error: 0.7519
Epoch 7/3000
21865/21865 [==============================] - 0s 1us/sample - loss: 0.7422 - mean_squared_error: 0.7422
Epoch 8/3000
21865/21865 [==============================] - 0s 1us/sample - loss: 0.7296 - mean_squared_error: 0.7296
Epoch 9/3000
21865/21865 [=============================

In [4]:
# import sklearn.metrics as metrics

# res_sorted = tf.sort(res)
# res_cdf = 1. * np.arange(res.shape[0]) / ((res.shape[0]) - 1)
# point_color = ["red" if label == 1 else "blue" for label in y_test]
# point_size = [0.1 if label == 0 else 10 for label in y_test]
# # plt.hist(res, bins=200)
# # plt.scatter(res, range(res.size), s=point_size, c=point_color)

# fpr, tpr, _ = metrics.roc_curve(y_test, res)

# precision, recall, threshold = metrics.precision_recall_curve(y_test, res)

# # precision = precision[0:-3]
# #
# # recall = recall[0:-3]

# plt.plot(recall, precision)

# # print(precision, recall)

# print(metrics.auc(fpr, tpr))
# # print(metrics.auc(precision, recall))

# # print(metrics.f1_score(test_labels, res))

# # plt.scatter(res_sorted, range(res.size), s=point_size, c=point_color)
# # plt.xlim(0, 150)
# plt.ylabel("precision")
# plt.xlabel("recall")
# plt.title("vanilla ae")
# plt.show()

In [5]:
# plt.plot(fpr, tpr)
# plt.xlabel("false positive")
# plt.ylabel("true positive")
# plt.title("vanilla ae")
# plt.show()

In [6]:
exp_size = 1000
exp_X = X_test[0:1]

In [11]:
# CEM
from alibi.explainers import CEM

# tf.compat.v1.disable_v2_behavior()
# tf.compat.v1.disable_eager_execution()

# class PredictFunc(nn.Module):
#     def __init__(self, the_model, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.ae_model = the_model
#         self.threshold = 65

#     def forward(self, x):
#         distance = train_model.autoencoder_test(x, x, self.ae_model, len(x))
#         res = 1 / (1 + np.exp(-(distance-self.threshold)))
        # return torch.tensor([[n] for n in res])

ae = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(np.size(data, axis=1),)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(inner_size, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(np.size(data, axis=1))
])

ae.compile(optimizer='adam', loss='mse', metrics=['mean_squared_error'])

# X_train, X_test = data[train_index], data[test_index]

ae.fit(X_train_raw, X_train_raw, epochs=3000, batch_size=np.size(data, axis=0))

# print(ae_model.predict(X_test[0:1]))

# @tf.function
def predFunc(x):
    # x = tf.cast(x, tf.float32)
    # distance = train_model.autoencoder_test(x, x, ae_model, len(x))
    # distance = tf.reduce_sum(tf.pow(ae_model(x) - x, 2), 1)
    # threshold = tf.constant(65, dtype=tf.float32)
    # res = 1 / (1 + tf.math.exp(-(distance-threshold)))
    # res = tf.reshape(res, (res.shape[0], 1))
    # # print(tf.concat([1-res, res], 1).shape)
    # return tf.concat([1-res, res], 1)
    # return tf.convert_to_tensor([[n] for n in res])
    pred = ae_model.predict(x)
    distance = np.power(pred - x, 2).sum(axis=1)
    threshold = 65
    res = 1 / (1 + np.exp(-(distance-threshold)))
    return np.array([[1-n, n] for n in res])

mode = 'PN'
shape = (1, ) + X_train.shape[1:]
feature_range = (X_train.min(axis=0).reshape(shape), 
                 X_train.max(axis=0).reshape(shape)) 
cem = CEM(predFunc, mode, shape, kappa=0., beta=1e-3, feature_range=feature_range, ae_model=ae, max_iterations=100, c_init=10, c_steps=10)

# cem.fit(X_train, no_info_type='median')

# exp_X_tensor = tf.convert_to_tensor(exp_X)
explanation = cem.explain(exp_X)


Train on 23457 samples
Epoch 1/3000


2023-08-30 10:52:12.055718: W tensorflow/c/c_api.cc:300] Operation '{name:'training_13/Adam/dense_42/bias/v/Assign' id:4934 op device:{requested: '', assigned: ''} def:{{{node training_13/Adam/dense_42/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training_13/Adam/dense_42/bias/v, training_13/Adam/dense_42/bias/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


23457/23457 [==============================] - 0s 8us/sample - loss: 0.9356 - mean_squared_error: 0.9356
Epoch 2/3000
23457/23457 [==============================] - 0s 3us/sample - loss: 0.9266 - mean_squared_error: 0.9266
Epoch 3/3000
23457/23457 [==============================] - 0s 2us/sample - loss: 0.9214 - mean_squared_error: 0.9214
Epoch 4/3000
23457/23457 [==============================] - 0s 1us/sample - loss: 0.9155 - mean_squared_error: 0.9155
Epoch 5/3000
23457/23457 [==============================] - 0s 1us/sample - loss: 0.9077 - mean_squared_error: 0.9077
Epoch 6/3000
23457/23457 [==============================] - 0s 2us/sample - loss: 0.8975 - mean_squared_error: 0.8975
Epoch 7/3000
23457/23457 [==============================] - 0s 1us/sample - loss: 0.8842 - mean_squared_error: 0.8842
Epoch 8/3000
23457/23457 [==============================] - 0s 1us/sample - loss: 0.8678 - mean_squared_error: 0.8678
Epoch 9/3000
23457/23457 [==============================] - 0s 1us/sa

No value specified for "no_info_val" through init or fit method.
NoneType: None


ValueError: 